In [ ]:
#Residue frequency (relative frequency)
from Bio import SeqIO
import pandas as pd
from collections import defaultdict

def calculate_residue_frequencies(reference_file, alignment_file, start_position, output_file="residue_frequencies.csv"):
    """Calculate the frequency of residues at each position across all sequences compared to the reference as percentages."""
    
    # Load the reference sequence
    reference = SeqIO.read(reference_file, "fasta")
    ref_seq = str(reference.seq).upper()  # Convert reference sequence to string and uppercase
    
    # Parse aligned sequences
    alignment = list(SeqIO.parse(alignment_file, "fasta"))
    
    # Dictionary to store residue frequencies at each position
    residue_freq = defaultdict(lambda: defaultdict(int))
    total_sequences = len(alignment)  # Total number of sequences
    
    # Iterate over each sequence in the alignment
    for record in alignment:
        seq = str(record.seq).upper()  # Convert sequence to string and uppercase
        
        # Iterate over each position in the sequence and reference
        for pos, (ref_base, seq_base) in enumerate(zip(ref_seq, seq), start=start_position):
            #if ref_base != '-' and seq_base != '-':  # Skip positions with gaps
                residue_freq[pos][seq_base] += 1  # Count the residue at this position
    
    # Convert the residue frequency dictionary to a DataFrame for easy viewing
    residues = []
    for pos, freq_dict in residue_freq.items():
        total_residues = sum(freq_dict.values())  # Total number of residues at this position (excluding gaps)
        
        # Calculate the percentage frequency of each residue
        residue_percentages = {residue: (count / total_residues) * 100 for residue, count in freq_dict.items()}
        residue_percentages["Position"] = pos  # Add the position number to the dictionary
        residues.append(residue_percentages)
    
    # Convert list of dictionaries to DataFrame
    residue_df = pd.DataFrame(residues).fillna(0)  # Fill missing values with 0
    print(f"Residue frequencies as percentages at each position:\n{residue_df}")
    
    # Save the frequency data to a CSV file
    residue_df.to_csv(output_file, index=False)
    print(f"Residue frequencies saved to {output_file}")
    
    return residue_df

# Example usage
reference_file = "REF.fasta"  # Path to the reference sequence file
alignment_file = "trimmed_sequences.fasta"  # Path to the aligned sequences file
output_file = "variant_frequency.csv"  # Output CSV file
start_position = xxx  # Please add the start position of your trimmed input relative to reference file
# Call the function to calculate residue frequencies and save them to a CSV file
residue_df = calculate_residue_frequencies(reference_file, alignment_file, start_position, output_file)


In [ ]:
###
import pandas as pd

# Read the input CSV file into a DataFrame
input_file = "variant_frequency.csv"  # Replace with your file path
df = pd.read_csv(input_file)

# Create a new column 'Percentage' to store the corresponding residue percentage
percentages = []

# Iterate over each row in the DataFrame
for _, row in df.iterrows():
    residue = row['Residue']  # Get the residue from the 'Residue' column
    percentage = row[residue]  # Look up the percentage from the corresponding residue column
    percentages.append(percentage)  # Add the percentage to the list

# Add the percentages to the DataFrame
df['Percentage'] = percentages

# Print the updated DataFrame
print(df)

# Save the updated DataFrame to a new CSV file
output_file = "updated_variant_frequency.csv"  # Replace with your desired output file path
df.to_csv(output_file, index=False)
print(f"Updated data saved to {output_file}")


In [ ]:
#5 to check the conseverd with threshold of 99.9
import pandas as pd

def analyze_conservation(input_csv, output_csv="conservation_analysis.csv"):
    """Analyze the conservation of each residue position based on the percentage frequencies."""
    
    # Load the residue frequencies CSV
    residue_df = pd.read_csv(input_csv)
    
    # List to store conservation results
    conservation_results = []
    
    # Iterate through each position and check the frequency percentages
    for idx, row in residue_df.iterrows():
        position = row['Position']
        is_conserved = "Conserved"
        
        # Check all columns (residues) for the current row
        total_residues = sum(row[1:])  # Sum of frequencies in all residue columns (excluding Position column)
        
        # If all frequencies for the position are zero, consider it as not conserved
        if total_residues == 0:
            is_conserved = "Not Conserved"
        
        # Check if any residue has 100% frequency at the current position
        for residue in residue_df.columns[1:]:  # Skip the 'Position' column
            if 100 > row[residue] > 99:
                is_conserved = "Not Conserved"
                break
        else:
            # If no residue has 100% frequency, mark as "Not Conserved"
            is_conserved = "Conserved"
        
        # Append the result for this position
        conservation_results.append({"Position": position, "Conservation Status": is_conserved})
    
    # Convert the results to a DataFrame
    conservation_df = pd.DataFrame(conservation_results)
    
    # Save the conservation analysis results to a new CSV file
    conservation_df.to_csv(output_csv, index=False)
    print(f"Conservation analysis saved to {output_csv}")
    
    return conservation_df

# Example usage
input_csv = "variant_frequency.csv"  # Path to the residue frequencies CSV
output_csv = "99perc_conservation_analysis.csv"  # Output CSV file for conservation analysis

# Call the function to analyze conservation and save the results
conservation_df = analyze_conservation(input_csv, output_csv)
print(conservation_df)
